In [1]:
import glob
import sys
import math
import numpy as np
import tensorflow as tf
from random import shuffle
from random import seed

from device import Device
from stroke import Stroke
from sample import Sample
from datetime import datetime
import pickle
import sklearn.utils
import statistics

In [2]:
class MySample:
    def __init__(self, events, sequence_counter, count, inch, jump):
        self.angle = 0
        self.inch = inch

        self.x = []
        self.y = []
        self.time = []
        for i in range(sequence_counter, sequence_counter+count):
            if i >= jump-1:
                self.x.append(events[jump-1][0])
                self.y.append(events[jump-1][1])
                
                if i == sequence_counter:
                    self.time.append(17)
                else:
                    self.time.append(events[jump-1][3]-events[jump-2][3])
            else:
                self.x.append(events[i][0])
                self.y.append(events[i][1])
                if i == sequence_counter:
                    self.time.append(17)
                else:
                    self.time.append(events[i][3]-events[i-1][3])

    def derivate(self):
        x_der = []
        y_der = []
        time_der = []
        for i in range(len(self.x)-1):
            x_der.append(self.x[i+1]-self.x[i])
            y_der.append(self.y[i+1]-self.y[i])
            time_der.append(self.time[i+1])
        self.x = x_der
        self.y = y_der
        self.time = time_der

    def getAngle(self, startIndex, endIndex):
        myX = self.x[endIndex]-self.x[startIndex]
        myY = self.y[endIndex]-self.y[startIndex]
        return math.atan2(myX, myY)

    def rotate(self, angle, originIndex):
        self.rotation = angle

        cs = math.cos(angle)
        sn = math.sin(angle)
        for i in range(len(self.x)):
            myX = (self.x[i]-self.x[originIndex]) * cs -\
                (self.y[i]-self.y[originIndex])*sn
            myY = (self.x[i]-self.x[originIndex]) * sn +\
                (self.y[i]-self.y[originIndex])*cs
            self.x[i] = myX + self.x[originIndex]
            self.y[i] = myY + self.y[originIndex]

In [3]:
def loadStudyData(path, inType, sampleLength):
    file_list = []
    name = "xxx"
    if inType == 1: name = "FittsTasks-participant"
    elif inType == 2: name = "PaintTasks-participant"
    elif inType == 3: name = "WriteTasks-participant"

    fileName = path+name

    for index in range(1,9):
        file_list.append(fileName+str(index)+".txt")

    samples = []

    for fileName in file_list:
        events = []
        f = open(fileName, 'r')
        for line in f:
            tokens = line.split(';')
            events.append([float(tokens[2]), float(tokens[3]), float(tokens[4]), int(tokens[1]), int(tokens[5])])

        seqCount = 0
        for i, event in enumerate(events):
            jump = i+sampleLength+1

            for j in range(i, i+sampleLength):
                if j >= len(events) or  events[j][4] is not 2:
                    jump = j
                    break
            if jump-i > 11:
                sample = MySample(events, i, sampleLength, 7, jump)

                sample.angle = sample.getAngle(9, 10) + math.radians(45)
                sample.rotate(sample.angle, 10)

                sample.derivate()

                sameTime = 0
                for a in range(len(sample.time)):
                    if sample.time[a] < 1:
                        sameTime = 1
                if sameTime == 0:
                    samples.append(sample)

    return samples

In [4]:
def buildModel(x, n_input, n_output, keep_prob):
    weights = {
        'h1': tf.get_variable('h1', shape=(n_input, 4096), initializer=tf.contrib.layers.xavier_initializer()),
        'h2': tf.get_variable('h2', shape=(4096, 2048), initializer=tf.contrib.layers.xavier_initializer()),
        'h3': tf.get_variable('h3', shape=(2048, 1024), initializer=tf.contrib.layers.xavier_initializer()),
        'h4': tf.get_variable('h4', shape=(1024, 512), initializer=tf.contrib.layers.xavier_initializer()),
        'out': tf.Variable(tf.random_normal([512, n_output]))
    }

    biases = {
        'b1': tf.Variable(tf.random_normal([4096])),
        'b2': tf.Variable(tf.random_normal([2048])),
        'b3': tf.Variable(tf.random_normal([1024])),
        'b4': tf.Variable(tf.random_normal([512])),
        'out': tf.Variable(tf.random_normal([n_output]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_1 = tf.nn.dropout(layer_1, keep_prob)

    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_2 = tf.nn.dropout(layer_2, keep_prob)

    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    layer_3 = tf.nn.relu(layer_3)
    layer_3 = tf.nn.dropout(layer_3, keep_prob)

    layer_4 = tf.add(tf.matmul(layer_3, weights['h4']), biases['b4'])
    layer_4 = tf.nn.relu(layer_4)
    layer_4 = tf.nn.dropout(layer_4, keep_prob)

    out_layer = tf.matmul(layer_4, weights['out']) + biases['out']
    return out_layer, weights

def buildPerceptron():
    learning_rate = tf.placeholder(tf.float32, shape=[])

    x = tf.placeholder("float", [None, 33])
    y = tf.placeholder("float", [None, 2])
    keep_prob = tf.placeholder(tf.float32)

    pred, weights = buildModel(x, 33, 2, keep_prob)

    cost = tf.reduce_mean(tf.pow(pred - y, 2))
    cost = cost + tf.sqrt(cost + 0.0000001)
    cost = tf.reduce_mean(cost +
        0.5*tf.nn.l2_loss(weights['h1']) +
        0.5*tf.nn.l2_loss(weights['h2']) +
        0.5*tf.nn.l2_loss(weights['h3']) +
        0.5*tf.nn.l2_loss(weights['h4']) +
        0.5*tf.nn.l2_loss(weights['out']))
    perf = tf.sqrt(tf.reduce_sum(tf.pow(pred - y, 2)))

    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

    # Initializing the variables
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()
    
    return init, saver, pred, x, y, keep_prob

def buildPerceptronVectors(samples, steps):
    inStudyVec = []
    outStudyVec = []
    
    for sample in samples:
        line = []
        for i in range(10):
            line.append(sample.x[i])
            line.append(sample.y[i])
            line.append(sample.time[i])

        myTime = 0
        for i in range(10, 10+steps):
            myTime = myTime + sample.time[i]

        line.append(myTime)
        line.append(sample.angle)
        line.append(sample.inch)

        inStudyVec.append(line)

        x = 0
        y = 0
        for i in range(0,steps):
            x = x + sample.x[10 + i]
            y = y + sample.y[10 + i]
        outStudyVec.append([x, y])

    inStudyVec = np.array(inStudyVec)
    outStudyVec = np.array(outStudyVec)

    return inStudyVec, outStudyVec

def getPerceptronPerformance(sess, pred, x, y, keep_prob, inVec, outVec):    
    batch_size = 500
    total_batch = int(len(inVec)/batch_size)
    avgDist = 0
    for i in range(int(len(inVec)/batch_size)+1):
        batch_x = inVec[i*batch_size:min((i+1)*batch_size, len(inVec))]
        batch_y = outVec[i*batch_size:min((i+1)*batch_size, len(inVec))]

        c = sess.run(pred, feed_dict={x: batch_x, y: batch_y, keep_prob: 1})
        for j in range(len(batch_x)):                
            dist = (batch_y[j][0]-c[j][0])*(batch_y[j][0]-c[j][0])
            dist = dist + (batch_y[j][1]-c[j][1])*(batch_y[j][1]-c[j][1])
            dist = math.sqrt(dist)
            avgDist = avgDist + dist
    return avgDist / len(inVec)

In [5]:
tasks = ["fitts", "draw", "write"]
time = ["Perceptron 33", "Perceptron 67", "Perceptron 100"]

init, saver, pred, x, y, keep_prob = buildPerceptron()
for j1, store in enumerate(time):
    j = (j1+1) * 2
    
    with tf.Session() as sess:
        sess.run(init)
        saver.restore(sess, "./models/"+store)
        avgError = 0
        for i in range(1,4):
            samples = loadStudyData('./data/', i, 11+j)
            inStudyVec, outStudyVec = buildPerceptronVectors(samples, j)
            
            perf = getPerceptronPerformance(sess, pred, x, y, keep_prob, inStudyVec, outStudyVec)
            avgError = avgError + perf

            print("{:.2f}".format(j*16.6666), "ms ", tasks[i-1], "{:.1f}".format(perf), "px")
        print("{:.2f}".format(j*16.6666), "ms ", "average", "{:.1f}".format(avgError/3), "px\n")

INFO:tensorflow:Restoring parameters from ./models/Perceptron 33
33.33 ms  fitts 8.7 px
33.33 ms  draw 14.3 px
33.33 ms  write 5.6 px
33.33 ms  average 9.5 px

INFO:tensorflow:Restoring parameters from ./models/Perceptron 67
66.67 ms  fitts 16.8 px
66.67 ms  draw 32.8 px
66.67 ms  write 13.0 px
66.67 ms  average 20.8 px

INFO:tensorflow:Restoring parameters from ./models/Perceptron 100
100.00 ms  fitts 27.8 px
100.00 ms  draw 55.4 px
100.00 ms  write 23.1 px
100.00 ms  average 35.4 px

